## Problem 2: The Superposed Walker

In [47]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator

### Parameters

In [48]:
steps=100                #no need for multiple walks however, as quantum walk is determinstic
world_size=2*steps+1     #as the farthest Bob could go from x=0 on any one side is equal to no. of steps

### Core Logic

In [ ]:
hadamard_circuit=QuantumCircuit(1)
hadamard_circuit.h(0)
hadamard_matrix=Operator(hadamard_circuit).data 
#we get the hadamard gate in the form of a matrix that we can directly apply

rms_displacement=[]
state=np.zeros((world_size,2))
#state[i,j] is the position amplitude to be at position i with coin in |j> where j=0,1
#amplitudes can have complex values too but since that's not possible in this problem, im not taking data type of matrix as complex

starting_point=steps                     #as x=0 is the row index 100 in the state matrix
state[starting_point,0]=1.0              #as Bob starts from x=0 with coin in |0> with absolute certainity

for t in range(steps):                          #everything must be iterated for each step

    for i in range(world_size):
        state[i]=hadamard_matrix@state[i]      
        #we apply the hadamard gate to each state vector of every position to get a new state vector

    new_state=np.zeros((world_size, 2))
    #new, blank state vector to store the changes of shift operation

    for i in range(world_size):
        #implementing the shift operations using a new matrix keeping in mind the indexing of the state matrix
        if i>0:
            new_state[i-1,0]+=state[i,0]      
        if i<world_size-1:
            new_state[i+1,1]+=state[i,1]
    
    state=new_state            #copying the new updated state into the original state matrix

    if t<3:                    #printing the position amplitudes for steps 1,2 and 3
        print(f"---------------- At step {t+1} ----------------")
        for i in range(world_size):
            amp0=state[i,0]
            amp1=state[i,1]
            prob=(np.abs(amp0)**2)+(np.abs(amp1)**2)

            if prob>0:
                position=i-starting_point
                if abs(amp0)>0:
                    print(f"Position: |x={position}>⊗|0> with amplitude = {amp0:.4f}")
                if abs(amp1)>0:
                    print(f"Position: |x={position}>⊗|1> with amplitude = {amp1:.4f}")
    
    probabilities=np.sum(np.abs(state)**2, axis=1)
    #calculating probability of each position by squaring and adding the amplitudes of |0> and |1> for each position giving a 1D array of 201 elements

    positions_squared=(np.arange(world_size)-starting_point)**2
    #the first step of calculating RMS, squaring the positions

    mean_squared=np.sum(probabilities*positions_squared)
    #mean-squared is equivalent to the expectation of positions squared, which is sum of P(x)x^2 where x is the position

    rms = np.sqrt(mean_squared)
    rms_displacement.append(rms)
    #appending the rms displacement till the i-th step


### Plotting graphs

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, steps + 1), rms_displacement, label="RMS Displacement")
plt.title("Quantum Walk: RMS Displacement")
plt.xlabel("Number of Steps (t)")
plt.ylabel("RMS Displacement")
plt.legend()
plt.grid(True)
plt.show()

### Q. Does this spread faster than the classical random walk? Why?

Yes, it is faster as it spreads linearly compared to the square-root spread of the classical version.

This increase is due to the combined effect of two principles: Superposition and Interference. 

The Hadamard ”coin” operator places the walker in a superposition of moving left and right simultaneously. This allows the ”edges” of the state’s wavefunction to always propagate outwards at the maximum possible speed (one step per time unit). Also, paths can have negative amplitudes in this case. When these paths recombine at later steps, they can cancel out (destructive interference) in the "middle" of the distribution, while constructively interfering at the "edges," pushing the walker away from the origin much faster than is possible classically.